In [2]:
# Import required libraries and dependencies.
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Retrieve, summarize and visualize data from crypto market csv file

In [3]:
# Load the csv data into a pandas dataframe from local "Resources" folder.
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display first 5 rows of dataframe.
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [4]:
# Generate summary statistics of dataframe.
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [39]:
# Use hvplot to plot data and see what's in pre-scaled dataframe.
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [8]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file.
normalized_data = StandardScaler().fit_transform(df_market_data)

# Create a dataframe with the scaled data.
df_market_normalized = pd.DataFrame(
    normalized_data,
    columns=df_market_data.columns)

# Copy the crypto coin names column from the original data.
# Do this by getting index from original dataframe.
df_market_normalized["coin_id"] = df_market_data.index

# Set the 'coin_id' column as index for normalized dataframe.
df_market_normalized = df_market_normalized.set_index("coin_id")

# Display first five rows of normalized dataframe.
df_market_normalized.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [11]:
# Create a list with the number of k-values from 1 to 11.
# Experiment with up to 10 clusters (NOTE: 11 is an exclusive cutoff, so it does goes to 10 and is cutoff at 11).
k = list(range(1, 11))

# Create an empty list to store the inertia values.
inertia = []

# Create a for loop to compute the inertia with each possible value of k.
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters.
# 2. Fit the model to the data using `df_market_normalized` dataframe.
# 3. Append the model.inertia_ to the inertia list.
# 4. Set random_state to 42 so we can replicate this model each time program runs.
for i in k:
    model = KMeans(n_clusters=i, random_state=42)
    model.fit(df_market_normalized)
    inertia.append(model.inertia_)

C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is kno

In [13]:
# Create a dictionary with the data generated using KMeans model to plot elbow curve.
elbow_data = {
    "k": k, # Values from 1 to 10 created by our list ranging from 1 to 11(exclusive).
    "inertia": inertia # Values from list inertia list filled up by for loop.
}

# Create a dataframe with the data to plot the elbow curve.
df_elbow = pd.DataFrame(elbow_data)

# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# Use elbow rule-of-thumb to observe where rate of decrease in inertia slows down.
elbow_line_plot = df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)
elbow_line_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  Using our elbow method , we can observe on this line chart that the rate of decrease in inertia slows down quite significantly at 4 clusters, plateauing out significantly afterwards. From this we can determine that 4 is the best practical value choice for `k`. Theoretically we could use 5, 6, or even a greater number for `k`, but this would likely be adding superfluous complexity to our model. Therefore, we are sticking with 4 as our value. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [15]:
# Initialize the K-Means model using the best value for k.
# From our elbow method, we have determined that 4 is our best value.
model = KMeans(n_clusters=4)

# Fit the K-Means model using the scaled data.
model.fit(df_market_normalized)

C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [16]:
# Predict the clusters to group the cryptocurrencies using the scaled data.
crypto_clusters = model.predict(df_market_normalized)

# Print the resulting array of cluster values.
print(crypto_clusters)

[1 1 3 3 1 1 1 1 1 3 3 3 3 1 3 1 3 3 1 3 3 1 3 3 3 3 3 3 1 3 3 3 2 1 3 3 0
 3 3 3 3]


In [17]:
# Make a copy of the normalized dataframe.
df_market_predictions = df_market_normalized.copy()

# Add a new column to dataframe with array of predicted clusters for the data.
df_market_predictions["crypto_cluster"] = crypto_clusters

# Display first five rows of sample data.
df_market_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,1
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,1
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,3
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,1


In [18]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the 'coin_id' column in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_clusters_plot = df_market_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="crypto_cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrencies Clusters",
)
crypto_clusters_plot

:NdOverlay   [crypto_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [21]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

# Use the PCA model on normalized dataframe with `fit_transform` to reduce to 
# three principal components.
crypto_pca_data = pca_model.fit_transform(df_market_normalized)

# View the first five rows of the data.
crypto_pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [22]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_model.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Aproximately 89 percent of the total variance can be explained by our three principal components.

In [24]:
# Create a new DataFrame with the PCA data.
df_market_pca = pd.DataFrame(
    crypto_pca_data, 
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original dataframe.
# Do this by getting index from original dataframe.
df_market_pca["coin_id"] = df_market_data.index

# Set the 'coin_id' column as index.
df_market_pca = df_market_pca.set_index("coin_id")

# Display first five rows of sample data.
df_market_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [27]:
# Create a list with the number of k-values from 1 to 11.
# Experiment with up to 10 clusters (NOTE: 11 is an exclusive cutoff, so it does goes to 10 and is cutoff at 11).
k = list(range(1, 11))

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters.
# 2. Fit the model to the data using `df_market_pca` dataframe.
# 3. Append the model.inertia_ to the inertia list.
# 4. Set random_state to 42 so we can replicate this model each time program runs.
for i in k:
    model = KMeans(n_clusters=i, random_state=42)
    model.fit(df_market_pca)
    inertia.append(model.inertia_)

C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is kno

In [30]:
# Create a dictionary with the data to plot the elbow curve.
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a dataframe with the data to plot the elbow curve.
df_elbow_pca = pd.DataFrame(elbow_data_pca)

# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

#### We answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** We can determine that 4 is the best value for `k` because that is where the rate of decrease of inertia slows.


* **Question:** Does it differ from the best `k` value found using the original data?

  * **Answer:** No, it is the same value we used with the uncompressed data suggesting that PCA has cleanly simplified the data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [33]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4)

# Fit the K-Means model using the PCA data
pca_model.fit(df_market_pca)

C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Karlschultzcomposer8\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [34]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_clusters_pca = pca_model.predict(df_market_pca)
# Print the resulting array of cluster values.
print(crypto_clusters_pca)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 2 1 0 0 3
 0 0 0 0]


In [35]:
# Create a copy of the DataFrame with the PCA data
market_pca_predictions = df_market_pca.copy()

# Add a new column to dataframe with data from predicted clusters.
market_pca_predictions["crypto_cluster"] = crypto_clusters_pca

# Display first five rows of sample data.
market_pca_predictions.head()

,PC1,PC2,PC3,crypto_cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,1
ethereum,-0.458261,0.458466,0.952877,1
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,1


In [36]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clusters_plot_pca = market_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="crypto_cluster",
    hover_cols=["coin_id"],
    title = "Cryptocurrencies Clusters Using PCA Data",
)
clusters_plot_pca

:NdOverlay   [crypto_cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, we visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [37]:
# Composite plot to contrast the Elbow curves
# Use '+' operator to print out both elbow cruves side by side
elbow_line_plot + elbow_plot_pca

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [38]:
# Composite plot to contrast the clusters
# Use '+' operator to print out both scatter plots side by side
crypto_clusters_plot + clusters_plot_pca

:Layout
   .NdOverlay.I  :NdOverlay   [crypto_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [crypto_cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### We answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Even after compressing dataset using PCA we can see that this data is highly clusterable and wants to cluster into 4 sets in a very similar way. Notice the two clusters of just one crypto coin in both plots: In both plots the two isolated crypto coins are 'ethlend' and 'celsius-degree-token'.